# Challenge HTTP

Ce challenge a pour but d'analysée et comprendre les trames **HTTP** lors d'une connexion.

Pour commencer, comme pour les challenges précédent, on importe la librairie "Scapy" qui sera uttile pour les programmes Python.

In [10]:
from scapy.all import *
http = rdpcap("www-total.pcapng")

## Introduction au HTTP

Comme pour les trames FTP et Telnet, Wireshark n'affiche pas toute les trames HTTP avec le nom "HTTP". Certaines sont afficher en TCP comme les SYN, SYN/ACK, ACK et FIN.

On doit donc récupérer toute les trames qui uttilise le port HTTP, soit ici le port par défault : **80**.

La commande est donc la suivante : `tcp.port == 80 || http`

![Trames](https://i.ibb.co/ksx0ckvM/Capture-d-cran-2025-06-01-010438.png)

## Observation des trames

Lorsque l'on regarde la capture, on obtient plein d'information comme notament l'ip du serveur (**10.2.12.4**) ou aussi l'ip du client (**10.2.4.3**).

On peut aussi obtenir des informations comme le login et le mot de passe de la connexion **HTTP**. Ils apparaîsent Dans la ligne "*Authorization: Basic*", plus précisément dans la sous-trames "*Credentials*" (appelée **Headers HTTP**) de la partie **HTTP** des trames.

![](https://i.ibb.co/FkFr3qfy/Capture-d-cran-2025-06-01-001335.png)



## Script Python

In [35]:
def http_pcapng(packets):
    for pkt in packets:
        # On s'assure que le paquet contient les couches IP, TCP et Raw (payload)
        if pkt.haslayer(IP) and pkt.haslayer(TCP) and pkt.haslayer(Raw):
            payload = pkt[Raw].load.decode('utf-8', errors='ignore')
            # On cherche dans le payload la ligne Authorization: Basic ...
            for line in payload.split('\r\n'):
                if line.lower().startswith('authorization: basic '):
                    # Extraction de la chaîne base64
                    b64_credentials = line.split(' ', 2)[2]
                    try:
                        decoded_bytes = base64.b64decode(b64_credentials)
                        decoded_str = decoded_bytes.decode('utf-8', errors='ignore')
                        # login:password
                        if ':' in decoded_str:
                            login, password = decoded_str.split(':', 1)
                            print(f"Login : {login}")
                            print(f"Mot de passe : {password}")
                            return
                    except Exception as e:
                        print(f"Erreur décodage : {e}")
                        continue

http_pcapng(http)

Login : donald
Mot de passe : P0utine


Le code analyse chaque paquet du fichier, si celui ci est un **HTTP** alors il continue son analyse. Il regarde si dans la data il a une ligne nomée "*Authorization: Basic*". Si c'est le cas, alors il vas décodée la ligne suivante et récupérée le login et le mot-de-passe (Contenu dans le "*Credentials: login:mdp*").

Voici maintenant le script sniffer **HTTP** qui fonctionne de la même mannière que le précédent.

In [ ]:
def http_sniffer(pkt):
    if pkt.haslayer(TCP) and pkt.haslayer(Raw):
        tcp_layer = pkt[TCP]
        if tcp_layer.dport == 80 or tcp_layer.sport == 80:  # HTTP traffic
            try:
                payload = pkt[Raw].load.decode('utf-8', errors='ignore')
            except:
                return

            # Cherche le header Authorization: Basic ...
            for line in payload.split('\r\n'):
                if line.lower().startswith('authorization: basic '):
                    b64_credentials = line.split(' ', 2)[2]
                    try:
                        decoded_bytes = base64.b64decode(b64_credentials)
                        decoded_str = decoded_bytes.decode('utf-8', errors='ignore')
                        if ':' in decoded_str:
                            login, password = decoded_str.split(':', 1)
                            print(f"Login : {login}")
                            print(f"Mot de passe : {password}")
                    except Exception as e:
                        print(f"Erreur décodage : {e}")

if __name__ == "__main__":
    print("Démarrage du sniffer HTTP ...")
    sniff(filter="tcp port 80", prn=http_sniffer, store=False)

Voilà pour le **HTTP**